# 创建 Amazon Comprehend 自定义实体识别器

本实验将提供在控制台或使用API的方式创建自定义实体识别器以及批量推理任务的方法

### 1. 导入必要的依赖包

In [ ]:
from datetime import datetime
import boto3
import json

### 2. 获取 AWS 账号 ID

In [ ]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
print("Your account id is {}".format(account_id))

### 3. 为实验创建存储桶（若在实验2中已经创建，则会输出桶已经存在） 

In [ ]:
bucket_name = "comprehend-labs" + account_id +  "-2"
print ("Bucket name used is " + bucket_name)
s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

if (s3.Bucket(bucket_name).creation_date is None):
    s3_client.create_bucket(Bucket=bucket_name)
    print ("Created bucket " + bucket_name)
else:
    print ("Bucket Exists")

### 4. 下载训练数据[entity list, docs]和测试数据，然后上传到s3 bucket。 

In [ ]:
s3_client = boto3.client('s3')
s3_entity_prefix = 'entity-training'
host_name = 'http://d1fjxffqn7wkdo.cloudfront.net'
!wget {host_name}/aws-offerings.csv
response = s3_client.upload_file('./aws-offerings.csv', bucket_name, "{}/aws-offerings.csv".format(s3_entity_prefix))

!wget {host_name}/aws-offerings-docs.txt
response = s3_client.upload_file('./aws-offerings-docs.txt', bucket_name, "{}/aws-offerings-docs.txt".format(s3_entity_prefix))
        
!wget {host_name}/aws-offerings-test.txt
response = s3_client.upload_file('./aws-offerings-test.txt', bucket_name, "{}/aws-offerings-test.txt".format(s3_entity_prefix))

### 5. 查看前20行数据

In [ ]:
!head -20 aws-offerings.csv

In [ ]:
!head -20 aws-offerings-docs.txt

### 6. 记录这些输出的参数，用于在控制台创建训练任务时提供

In [ ]:
print("Entity List Location:  s3://{}/{}/aws-offerings.csv".format(bucket_name,s3_entity_prefix))
print("Training Documents Location:  s3://{}/{}/aws-offerings-docs.txt".format(bucket_name,s3_entity_prefix))
print("Test Documents Location:  s3://{}/{}/aws-offerings-test.txt".format(bucket_name,s3_entity_prefix))
print("Bucket Path:  s3://{}".format(bucket_name))

# 现在可以返回查看Workshop手册，根据手册中提供的步骤在控制台创建训练任务

---

## 下面是通过API创建训练任务和推理任务的步骤，供大家进行参考

### 1. 获取在上一个实验中创建的角色的 ARN 

In [ ]:
#This name should match the name of the role that was created in the first lab.
role_name_base = 'AmazonComprehendServiceRoleS3FullAccess-ComprehendLabs'
prefix_random_numbers = '' #If you added random numbers to the end of the 'ComprehendLabs' prefix, put them here
if not prefix_random_numbers:
    role_name = "{}{}".format(role_name_base,prefix_random_numbers)
else:
    role_name = role_name_base
iam_client = boto3.client("iam")
response = iam_client.get_role(
    RoleName=role_name
)
comprehend_arn = response['Role']['Arn']
print("The ARN for the role is {}".format(comprehend_arn))

### 2. 创建训练任务

In [ ]:
comprehend_client = boto3.client("comprehend")
response = comprehend_client.create_entity_recognizer(
    RecognizerName="Recognizer-Name-Goes-Here-{}".format(datetime.now()).replace(' ','-').replace(':','-').replace('.','-'),
    LanguageCode="en",
    DataAccessRoleArn=comprehend_arn,
    InputDataConfig={
        "EntityTypes": [
            {
                'Type': "AWS_OFFERING"
            }
        ],
        'EntityList': {
            'S3Uri': "s3://{}/{}/aws-offerings.csv".format(bucket_name,s3_entity_prefix)
        },
        'Documents': {
            'S3Uri': "s3://{}/{}/aws-offerings-docs.txt".format(bucket_name,s3_entity_prefix)
        },
        
    }
)
recognizer_arn = response["EntityRecognizerArn"]
print("The ARN for the entity recognizer is {}".format(recognizer_arn))

### 3. 检查训练任务的状态

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
#The possible statuses for the custom entity recognizer are: 'SUBMITTED'|'TRAINING'|'DELETING'|'STOP_REQUESTED'|'STOPPED'|'IN_ERROR'|'TRAINED'
print("The status of the custom entity recognizer is {}".format(response['EntityRecognizerProperties']['Status']))

### 4. 查看训练相关指标

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
if response['EntityRecognizerProperties']['Status'] == 'TRAINED':
    print(json.dumps(response['EntityRecognizerProperties']['RecognizerMetadata'], indent=2))
else:
    print ("Training job has not completed yet.  Please wait to check training performance until it has.")

### 5. 创建批量转化作业

In [ ]:
response = comprehend_client.describe_entity_recognizer(
    EntityRecognizerArn=recognizer_arn
)
if response['EntityRecognizerProperties']['Status'] == 'TRAINED':
    response = comprehend_client.start_entities_detection_job(
        JobName='AWS_OFFERING-001',
        EntityRecognizerArn=recognizer_arn,
        LanguageCode="en",
        DataAccessRoleArn=comprehend_arn,
        InputDataConfig={
            'S3Uri': "s3://{}/{}/aws-offerings-test.txt".format(bucket_name,s3_entity_prefix),
            'InputFormat': 'ONE_DOC_PER_LINE'
        },
        OutputDataConfig={
            'S3Uri': "s3://{}/{}/results/".format(bucket_name,s3_entity_prefix)
        }
    )
    job_id = response['JobId']
else:
    print ("Training job has not completed yet.  Please wait to start batch entity recognitino job until it has.")

### 6. 检查批量转换作业的状态 

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
print("The status of the batch entity detection job is {}".format(response['EntitiesDetectionJobProperties']['JobStatus']))

### 7. 下载批处理作业的输出

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    output_s3_uri = response['EntitiesDetectionJobProperties']['OutputDataConfig']['S3Uri']
    s3_key = output_s3_uri.replace("s3://{}/".format(bucket_name),'')
    s3.meta.client.download_file(bucket_name, s3_key, 'output.tar.gz')
    !tar zxvf output.tar.gz
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")

### 8. 让我们回顾一下测试数据并查看输出结果

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    !head -20 aws-offerings-test.txt
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")

In [ ]:
response = comprehend_client.describe_entities_detection_job(
    JobId=job_id
)
if response['EntitiesDetectionJobProperties']['JobStatus'] == "COMPLETED":
    !cat output
else:
    print("Batch transformation job not complete.  Please wait until this job is completed before attempting to view output.")